# More filters: the median filter and non-local filters

This page will cover filters which do not use convolution kernels. First, we will look at the median filter, which differs in important ways from the other local filters we saw on [previous](5_mean_filter) pages, because it cannot be implemented through convolution.

After the median filter, we will look at another filter, the histogram equalisation filter, which does not use convolution. In fact, histogram equalisation does not use a kernel at all. Because it eschews kernels completely, the histogram equalisation filter is a *non-local* filter. More on this below.

As normal, we begin with some imports:

In [ ]:
# Library imports.
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import skimage as ski
from mpl_toolkits.mplot3d import Axes3D

# Set 'gray' as the default colormap
plt.rcParams['image.cmap'] = 'gray'

# Set NumPy precision to 2 decimal places
np.set_printoptions(precision=2)

#  A custom function to quickly report image attributes.
from show_attributes import show_attributes

# Local filtering without convolution

When compared to the other local filters we have seen, the median filter uses the same process of "walking" through the image with a small array which denfines the "local neighbourhood" of each pixel. In median filtering, you can *loosely* think of this small array as a kernel, but there are some major differences in the calculations, so let's call it the "neighbourhood array" instead. 

Like with the other kernels we have seen, the "neighbourhood array" is centered on every pixel during the filtering operation. The central pixel value is replaced with a number computed from the other values in the local neighbourhood, e.g. the other pixel values under the "neighbourhood array".  However, instead of *convolving* the neighbourhood array and the image, as we would for a mean filter or Gaussian filter etc., the median filter, shockingly, takes the median value of the pixels under the small array. 

![](images/kernel_general.png)

There is no convolvable function to calculate the median, and this is why [some people](https://dsp.stackexchange.com/questions/13211/convolution-kernels-for-image-filtering) will tell you not to call the "neighbourhood array" a "kernel", when we are doing median filtering. Calculating the median utilizes ranking and indexing, and cannot be expressed as set of additions, subtractions, multiplications, divisions etc, in the way that calculating a mean can, for instance.

One way to think of this, is that when we write the formula for the mean, we can write it without reference to any Python indexing operations:

$ \large \text{mean} = \frac{\sum X}{n} $

...we can read that as "to get the mean of a set of numbers (where $X$ refers all of the numbers), add them up and divide by however many numbers there are ($n$)". 

Conversely for the median we first have to order the numbers from lowest to highest ($X_{\text{sorted}}$), and then find the value that is *at the central index*. So where $X$ is an array containing all the numbers, if $n$ is odd:

$ \large \text{median} = X_{\text{sorted}}[\frac{n}{2}]$

Where the square brackets are a *Python* indexing operation (we will need a different indexing operation if using a programming language that does not count indexes from 0, or if $n$ is even...). So for the numbers in the array below:

In [ ]:
# Some numbers.
nums = np.array([10, 4, 5, 8, 7])

nums

We get the mean from: $ \large \text{mean} = \frac{\sum x_i...x_n}{n} $

In [ ]:
# Take the sum, divide by n.
np.sum(nums)/len(nums)

In [ ]:
# Compare to the same calculation from NumPy.
nums.mean()

Whereas we get the median from: $ \large \text{median} = X_{\text{sorted}}[\frac{n}{2}]$

In [ ]:
# Get the median.
sorted_nums = np.sort(nums) # Sort the values, low to high.
print(f"Sorted `nums`: {sorted_nums}")
median = sorted_nums[int(len(sorted_nums)/2)] # Index to get the median.

# Show the median value.
median

In [ ]:
# Compare to NumPy.
np.median(nums)

There is no convolution kernel that can do this - instead we use the "neighbourhood array" to define a pixel neighbourhood, then replace the central value with the median from that neighbourhood, using the sorting and indexing we have seen above. So, the median filter *is* a local filter, because it alters pixel values based on other pixel values in the local neighbourhood. However, it is not a *convolution filter* because it does use the local neighbourhood in a convolution operation, in contrast to other local filters like the mean filter, Gaussian filter etc. 

# Edge preservation

The median filter is especially useful for removing noise from images, while preserving the "edges" in the image. You'll recall that the edges are big changes in the gradient of pixel intensities, between nearby pixels (e.g. black-to-white, white-to-black etc). Let's look at why the median filter is "edge preserving", by expanding the `nums` array into a low-resolution image, using `np.tile()` and `.reshape()`.

In [ ]:
# Make `nums` into a image array.
nums_img = np.tile(nums, reps=3).reshape((3, 5))
nums_img 

In [ ]:
# Show as an image.
plt.matshow(nums_img);

Now, imagine we "walk" a 3-by-3 kernel (sorry pedants!) through the `nums_img` array, and replace the central pixel of each kernel (sorry again, pedants!) with the median of the pixels under the kernel.

This process is shown below, for three kernels (OK, pedants, three "local neighbourhoods under a 3-by-3 array very much resembling a kernel"). The central pixel is highlighted in red, and the index of the current local neighbourhood is shown above the pixel values.

The flattened and sorted values from each local neighbourhood are also shown, along with the median value of the neighbourhood:

![](images/median_filter.png)

We can also show this in "Python space", using a for loop:

In [ ]:
# Show some local neighbourhoods of `nums_img` and their medians.
for i in np.arange(3):
    i_row_start, i_col_start = 0, i
    i_row_end, i_col_end = 3, i+3
    print(f"\nnums_img[{i_row_start}:{i_row_end}, {i_col_start}:{i_col_end}]")
    current_selection = nums_img[i_row_start:i_row_end, i_col_start:i_col_end]
    print(current_selection)
    print(f"Flattened and sorted: {np.sort(current_selection.ravel())}")
    print(f"Median = {np.median(nums_img[i_row_start:i_row_end, i_col_start:i_col_end])}")

Because the median is not a function which can be applied via convolution, we cannot apply it using `ndi.correlate()`, as we did with the other [local filters](5_mean_filter). 

We can apply it in `skimage` using `ski.filters.median()`. Again,  we supply a `footprint` argument to determine the size of each pixel's "local neighbourhood":

In [ ]:
# Median filter `nums_img`.
nums_median_filtered = ski.filters.median(nums_img, 
                                          footprint=np.ones((3,3)))
nums_median_filtered

Compare to the original `nums_img` array below:

In [ ]:
nums_img

The effect of the "edges" of the image is easier to see graphically:

In [ ]:
# Show the images.
plt.subplot(1, 2, 1)
plt.imshow(nums_img)
plt.title('Original')
plt.subplot(1, 2, 2)
plt.imshow(nums_median_filtered)
plt.title('Median Filtered');

Edges involving a bigger gradient have been preserved, whilst less prominent edges have been merged into more prominent ones. We show this, for comparison, alongside a mean filtered version of `nums_img`, filtered using the same size `footprint` ((3, 3)):

In [ ]:
nums_img = ski.util.img_as_ubyte(nums_img) # Avoid `dtype` warning.
nums_mean_filtered = ski.filters.rank.mean(nums_img,
                                           footprint=np.ones((3,3)))
# Show the images.
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(nums_img)
plt.title('Original')
plt.subplot(1, 3, 2)
plt.imshow(nums_median_filtered)
plt.title('Median Filtered')
plt.subplot(1, 3, 3)
plt.imshow(nums_mean_filtered)
plt.title('Mean Filtered');

You can see that the median filter gives a better "summary" of the distribution of edges in the original image - the mean filter has spread the edges around more, which makes sense, given that it averages pixels within a kernel. 

In a higher resolution image, the median filter can have the effect of removing noise whilst preserving edges to a greater extent than some other filters. We will demonstrate the median filter again with the `brick` image from `ski.data`:

In [ ]:
# Load in the `brick` image.
brick = ski.data.brick()
show_attributes(brick)
plt.imshow(brick);

We will also filter `brick` using a mean filter, with the same size kernel as the median filter:

In [ ]:
# Apply a median filter.
median_filtered_brick = ski.filters.median(brick, 
                                           footprint=np.ones((9,9)))

mean_filtered_brick = ski.filters.rank.mean(brick, 
                                            footprint=np.ones((9,9)))
# Plot both image to compare
plt.figure(figsize=(14, 4))
plt.subplot(1, 3, 1)
plt.title('Original Image')
plt.imshow(brick)
plt.subplot(1, 3, 2)
plt.title('Median Filtered')
plt.imshow(median_filtered_brick)
plt.subplot(1, 3, 3)
plt.title('Mean Filtered')
plt.imshow(mean_filtered_brick);

You can see that the edges in the images (transitions between pixels of very different intensities - in this case between the dark bricks and the lighter mortar lines) are less smoothed by the median filter than by the mean filter. This is considered a desirable property of the median filter.

# Non-local filters

Whilst the median filter does not use convolution, it is still a local filter, because it filters pixels based on the values of other pixels in the local neighbourhood. Other filters use neither convolution nor a local pixel neighbourhood. These filters are called *non-local filters*. 

Non-local filters have their name because they filter all of the pixels in an image based on characteristics of a specific region of the image, or based upon characteristics of the entire image. As a result, a non-local filter might modify a given pixel's value based on the values of pixels in a region of the image which is nowhere near the "local neighbourhood" of the pixel being modified.

One foundational non-local filter is a [*histogram equalisation filter*](https://en.wikipedia.org/wiki/Histogram_equalization). This filter modifies pixels based on the histogram of the entire image. Essentially, this process "smoothes out" the histogram into a even "hill", so that there is less variance between the pixel intensities. The image below illustrates this principle:

![](images/hist_equal.png)

We will demonstrate the histogram equalisation filter with the `eagle` image from `ski.data`:

In [ ]:
# Load in the `eagle` image.
eagle = ski.data.eagle()
show_attributes(eagle)
plt.imshow(eagle);

Let's first view the histogram of `eagle`, before we apply the filter. As we know, we can use the `.ravel()` array method to flatten this 2D image to 1D, and then inspect a histogram of the pixel intensities:

In [ ]:
# Flatted to 1D.
one_D_eagle = eagle.ravel()

# Show a histogram.
plt.hist(eagle.ravel(), 
         bins=128)
plt.xlabel('Pixel Intensity')
plt.ylabel('Pixel Count');

In order to apply the histogram equalisation filter, we first "deconstruct" into separate arrays using `np.histogram()`. This returns two arrays. One array we will call `counts`; this contains the height of the histogram within each $x$-axis bin. The second array we will call `bin_intervals`; adjacent values in this array are the start and end points of each $x$-axis bin. We use `bin_intervals` to calculate the centerpoint of each bin, by taking the average of the adjacent values.

*Note*: alternatively we could use `ski.exposure.histogram()`, to the same effect.

In [ ]:
# Centers and bin intervals, from the histogram of the flattened `eagle` image.
counts, bin_intervals = np.histogram(one_D_eagle,
                                     bins=256)

# Calculate the bin centers.
bin_centers = (bin_intervals[1:] + bin_intervals[:-1]) / 2

# Show the `counts` and `bin_centers`.
print(f"\nCounts:\n {counts}")
print(f"\nBin centers:\n {bin_centers}")

We chose to use 256 bins, for this histogram, for reasons that will become apparent further down the page (remember this!).

There are several steps in equalising the histogram:

- First, we normalise the histogram, so that the counts sum to 1. We do this by dividing each count by the total number of pixels in the image, which converts each count to a proportion.

- Then we calculate the *cumulative density* of the normalised histogram. Basically, this means we add up the proportions as we go along the $x$-axis, so each bin indicates the total proportion of pixels up to that point (e.g. pixels in that particular bin, or bins situated lower down the $x$-axis). 

- We then "map" each pixel intensity value to its corresponding cumulative proportion. After this "mapping", we have our equalised histogram.

This may all sound quite abstract, but bear with us. It is easier to follow in code, and is a visually striking effect when we compare the resulting histogram to the original. 

For the first step, we can normalise the `counts` by dividing each individual count by the total number of pixels in the image. Note that we could also do this using the optional `density=True` argument to `np.histogram()`, but we do it manually to show what the actual operations involve:

In [ ]:
# Centers and bin intervals, from the histogram of the flattened `eagle` image.
n_pixels = len(one_D_eagle) # Get the total number of pixels.
counts_normed = counts/n_pixels # Normalize by dividing each count by the total number of pixels.
plt.plot(bin_centers, counts_normed)
plt.xlabel('Pixel Intensity')
plt.ylabel('Pixel Probability');

In [ ]:
# Do the `counts` sum to 1?
print(f"\nCounts Normalized sum:\n {counts_normed.sum()}")

When we equalise the histogram, we want it to be roughly uniform across the pixel intensities. As a tool to perform this "uniformization", we first we calculate the cumulative distribution (`cdf`) of the histogram. To do this we take cumulative sum of the normalised counts (`counts_normed`). For a given bin, the cumulative sum operation adds up the proportion of pixels that are in that bin and in all of the lower bins (e.g. the bins closer to 0 on the $x$-axis). Essentially it is a running total of the `counts_normed` as we move from left to right across the $x$-axis:

In [ ]:
# Get the cumulative distribution of the pixel intensities.
cdf = counts_normed.cumsum() 

# Show a plot of the cumulative distribution.
plt.plot(bin_centers, cdf)
plt.xlabel('Pixel Intensity')
plt.ylabel('Cumulative Proportion');

The final value in `cdf` is 1 (or at least damn near close, given precision loss in the calculations). Remember, we are adding up proportions here, so a value of 1 indicates that the final bin and all of the lower bins together contain 100% of the pixels in the image:

In [ ]:
cdf[-1]

The next step requires some thought, to follow what is going on. We want to "map" the pixel values in the flattened `one_D_eagle` array to the values in the `cdf`. We can do this, for the current `eagle` image, by using the `one_D_eagle` pixel intensity values as *indexes* for the `cdf` array. This might seem like a hack, so lets break it down.

Recall that we asked you to remember that we asked `np.histogram()` to give us a histogram with 256 bins? Well, as a result our `cdf` array, which contains the running total of the proportions in a given bin and lower bins, has 256 elements:

In [ ]:
# Show the `shape` of the `cdf` array.
cdf.shape

Because we count from 0, when indexing arrays, the final value in `cdf` is at index location 255:

In [ ]:
# Show the final value of the `cdf` array, at integer index location 255.
cdf[255]

Given that `one_D_eagle` is in the `uint8` `dtype`, the maximum value allowed is 255, and the minimum is 1:

In [ ]:
# Show the `dtype` and `min`/`max` values of `eagle`.
show_attributes(one_D_eagle)

Conveniently here, each pixel intensity value in `one_D_eagle` will work as an index into `cdf`. This allows us to replace each pixel intensity value with its corresponding cumulative proportion. So, if a pixel intensity value $p$ falls in bin $b$, then bin $b$ has a corresponding cumulative proportion in the `cdf` array. We use this "mapped" values as our output image - this has the effect of "smoothing out", or, in fact, "equalising" the histogram, because bins without many pixels in them "inherit" the proportions from lower bins. Again, this is easier to appreciate visually, by viewing the histograms below.

Let's perform the indexing/equalisation, and then inspect the histograms, so this effect becomes apparent. We show this mapping first with ten pixel intensity values from `one_D_eagle`:

In [ ]:
first_ten_pixels = one_D_eagle[:10]
first_ten_pixels

...we then show the corresponding cumulative proportions that these values will be mapped to, when we use them as indexes for `cdf`:

In [ ]:
cdf[first_ten_pixels]

To perform this mapping for *every* pixel intensity value, we use the following operation:

In [ ]:
# Equalise the histogram of `eagle`, by using the pixel intensity values in `one_D_eagle` (1 - 255)
# as indexes into the `cdf` array.
equalised_hist = cdf[one_D_eagle]

The original histogram and the equalised histogram, along with the corresponding images, are shown below. We first `.reshape()` the `equalised_hist` back into the shape of the original, non-flattened `eagle` image, thus restoring its status as a 2D image array:

In [ ]:
# Reshape to 2D.
eagleback_to_2D = equalised_hist.reshape(eagle.shape)

# Generate the plot.
plt.figure(figsize=(14, 8))
plt.subplot(2, 2, 1)
plt.imshow(eagle)
plt.title('Original Image')
plt.subplot(2, 2, 2)
plt.title('Original Histogram')
plt.hist(eagle.ravel(), bins=128)
plt.subplot(2, 2, 3)
plt.imshow(eagleback_to_2D)
plt.title('Equalised Image')
plt.subplot(2, 2, 4)
plt.title('Equalised Histogram')
plt.hist(eagleback_to_2D.ravel());

The equalised histogram now much more closely resembles a [uniform distribution](https://en.wikipedia.org/wiki/Uniform_distribution) - the notable "peaks and valleys" of the original and been replaced with a uniform block. The effect on the perceived visual image is one of heightened contrast - pay particular attention to the wall behind the noble eagle. Each image, without the histograms, can be viewed below, for easier inspection:

In [ ]:
# Generate the plot.
plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.imshow(eagle)
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(eagleback_to_2D)
plt.title('Equalised Image');

Obviously, this operation has changed the `dtype` of the image because we have replaced values ranging from 1 to 255 with proportions ranging from 0 to 1:

In [ ]:
# The `dtype` and `min`/`max` values of the original image.
show_attributes(eagle)

In [ ]:
# The `dtype` and `min`/`max` values of the filtered image.
show_attributes(equalised_hist)

This is important to be aware of, but we can easily convert back to the original `dtype` using the relevant function from `ski.util`. Speaking of `ski`, les look at how to apply this filter in `skimage` in the next section...

# Histogram equalisation in `skimage`

As normal, `skimage` makes it easy to implement this filter, in just a single line of code. We just pass our `eagle` image to the `ski.exposure.equalize_hist()` function, and it will carry out all we saw above, behind the scenes:

In [ ]:
# Equalise `eagle` using `skimage.
eagle_equalized_with_ski = ski.exposure.equalize_hist(eagle)

# Generate the plot.
plt.figure(figsize=(14, 8))
plt.subplot(2, 2, 1)
plt.imshow(eagle)
plt.title('Original Image')
plt.subplot(2, 2, 2)
plt.hist(eagle.ravel(), bins=128)
plt.title('Original Histogram')
plt.subplot(2, 2, 3)
plt.imshow(eagle_equalized_with_ski)
plt.title('Equalised Image (via `skimage`)')
plt.subplot(2, 2, 4)
plt.hist(eagle_equalized_with_ski.ravel())
plt.title('Equalised Histogram (via `skimage`)')
plt.tight_layout();

Easy peasy. If you consult the [documentation](https://scikit-image.org/docs/0.25.x/api/skimage.exposure.html#skimage.exposure.equalize_hist) for `ski.exposure.equalize_hist()`, you notice that by default, it uses 256 bins. In fact, for the `nbins` optional argument, the documentation states that it will be ignored for integer data. This is so the "indexing trick" that we saw above can be performed. 

It is no problem to use this filter with `float` image data however - the principle is the same pixel intensities get mapped to their corresponding cumulative proportion. Only now this occurs without the neat indexing trick, there is just an intermediate (boring!) step in the mapping. We demonstrate this below with the `coins` image from `ski.data`:

In [ ]:
# Import the `coins` image.
coins = ski.data.coins()

# Convert to `float64` `dtype`.
coins_as_float = ski.util.img_as_float64(coins)

# Show the image, the histogram of the image, and the attributes of the image.
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(coins_as_float)
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.hist(coins_as_float.ravel())
plt.title('Original Histogram')
plt.tight_layout();
show_attributes(coins_as_float)

In [ ]:
# Equalise the histogram.
coins_as_float_equalised_with_ski = ski.exposure.equalize_hist(coins_as_float)

# Show the image, the histogram of the image, and the attributes of the image.
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(coins_as_float_equalised_with_ski)
plt.title('Equalised Image (via `skimage`)')
plt.subplot(1, 2, 2)
plt.hist(coins_as_float_equalised_with_ski.ravel())
plt.title('Equalised Histogram (via `skimage`)')
plt.tight_layout();
show_attributes(coins_as_float_equalised_with_ski)

# Summary

This page has shown a local filter (the median filter) and a non-local filter (histogram equalisation) which do not use convolution to perform their filtering operations. On the [next page](8_morphology) we will introduce another method for modifying pixels based on a `footprint`.

# References

3.3.5 onward from: https://lectures.scientific-python.org/packages/scikit-image/index.html

Based on: https://scikit-image.org/skimage-tutorials/lectures/1_image_filters.html

Histogram equalisation adapted from: https://www.janeriksolem.net/histogram-equalization-with-python-and.html

Histogram equalisation: https://medium.com/jungletronics/histogram-equalization-34149fc299a6

Reference: https://www.kdnuggets.com/numpy-for-image-processing

Reference: https://setosa.io/ev/image-kernels

Reference: https://wiki.imindlabs.com.au/ds/aml/4_problem_domains/1-image-processing/3_edge_detectors

Reference: https://www.geeksforgeeks.org/deep-learning/types-of-convolution-kernels

Reference: skimage tutorials (check versions), scipy lecture notes

Reference: https://jni.github.io/i2k-skimage-napari/lectures/1_image_filters.html